<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [9]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)


In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
cp = pd.read_csv('/home/bioinformatica/Descargas/listado-de-codigos-postales-de-españa.csv', 
                 encoding='latin-1', sep=';',
                 dtype={'codigopostalid': object})

In [171]:
cp['cp'] = cp.codigopostalid.apply(lambda x: f'{x:0>5}')

In [207]:
len(cp['codigopostalid'])

14665

In [6]:
base_url = 'https://servicio.mapama.gob.es/regoe/Publica/Operadores2.aspx'

In [8]:
provincia = '01'

In [2]:
k = 2

In [4]:
f'{k:0>2}'

'02'

In [ ]:
params = {
    __EVENTTARGET=&
    __EVENTARGUMENT=&
    __LASTFOCUS=&
    __VIEWSTATE=%2FwEPDwULLTIwMjA4NzA3MjQPFhIeDVNlbGVjY2lvbmFibGUFLENUaDFKMVlXQVV0WlRaZDhRNHBIRUNZcnFSeE1ZNHZ0R2o4SXI1M2wyM1k9HglBY3RpdmlkYWQFLHlSV1Fhb1I5UllJbWdVVituMGo2bWtFdU10dUNITjFZRko1MXc2TFB4a2M9Hg9BY3RpdmlkYWROaXZlbDUFLEIzTmRHa1lNRnkxVmdQc2tHTDdTVE50U2JHZ0VYNlJKZzY5K0VDQ214VVE9Hg9BY3RpdmlkYWROaXZlbDQFLEV6a3I0aTZiNGljTlpLdXVFa2thNWlBSTBNMTBqZGlGYzVwNGFVZlNyS289HhRDbGF2ZVN1YlRpcG9PcGVyYWRvcgUsK2J3MjZzQWpaeE1BTS9jU1dIVGorTkI1cW1CMjNGYSt1M2I4RlBtZm9LRT0eD0FjdGl2aWRhZE5pdmVsMQUsSTlMZjJnYVBsQndVRFo3UDNQVUVscGllaUc0R1NRdUtvWXR2Z0FhOVc4Zz0eD0FjdGl2aWRhZE5pdmVsMwUsMmcxKzh4bjRWTS9FVy8rTy9Fd0xUZzFUY0NYVUU1MGdPb1Vjb1hjRzMvOD0eD0FjdGl2aWRhZE5pdmVsMgUsL3BCNG82UDg1b0l3Q1hnMzYzdU5SdHlGbjJicG9JS0NGQ0VVcVI4ZXloND0eDkNsYXZlQWN0aXZpZGFkBSxjeWp4QTlUN2JydEo5SHVicXVvelJ1Vk02SVhPM0Q1V0x1QWExa1VwcnFzPRYCZg9kFggCAg8PFgIeBFRleHQFHkVycm9yIHJlY3VwZXJhY2nDs24gdXN1YXJpby4gM2RkAgMPDxYEHwllHgdWaXNpYmxlaGRkAgQPDxYCHgVHcnVwbwUsSnkrSUpyTW9nbzJKMGEyaFBSdTJhVjhRSU4vMk9sdGxtSDJuQjJHZTZraz1kFgJmD2QWAmYPFgIfCQWnATxsaSBjbGFzcz0nb3Blbic%2BPGgyPjxhIGhyZWY9J2h0dHBzOi8vc2VydmljaW8ubWFwYW1hLmdvYi5lcy9yZWdvZS9QdWJsaWNhL09wZXJhZG9yZXMuYXNweCcgaWQ9T3BjaW9uMD5MaXN0YWRvIGRlIE9wZXJhZG9yZXMgZGUgbGEgQWdyaWN1bHR1cmEgRWNvbMOzZ2ljYTwvYT48L2gyPjwvbGk%2BZAIFD2QWAgIBD2QWBgIBD2QWGAIPDxAPFgIeB0NoZWNrZWRoZGRkZAITDxAPFgYeDkRhdGFWYWx1ZUZpZWxkBQVDbGF2ZR4NRGF0YVRleHRGaWVsZAUGTm9tYnJlHgtfIURhdGFCb3VuZGdkEBUUBVRvZGFzCkFuZGFsdWPDrWEHQXJhZ8OzbhdBc3R1cmlhcywgUHJpbmNpcGFkbyBkZQ5CYWxlYXJzLCBJbGxlcwhDYW5hcmlhcwlDYW50YWJyaWEUQ2FzdGlsbGEgLSBMYSBNYW5jaGEQQ2FzdGlsbGEgeSBMZcOzbglDYXRhbHXDsWEFQ2V1dGEUQ29tdW5pdGF0IFZhbGVuY2lhbmELRXh0cmVtYWR1cmEHR2FsaWNpYRRNYWRyaWQsIENvbXVuaWRhZCBkZQdNZWxpbGxhEk11cmNpYSwgUmVnacOzbiBkZRtOYXZhcnJhLCBDb211bmlkYWQgRm9yYWwgZGULUGHDrXMgVmFzY28JUmlvamEsIExhFRQBMAExATIBMwE0ATUBNgE4ATcBOQIxOAIxMAIxMQIxMgIxMwIxOQIxNAIxNQIxNgIxNxQrAxRnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZxYBZmQCFw8QDxYGHw0FBUNsYXZlHw4FBk5vbWJyZR8PZ2QQFTUFVG9kYXMIQWxiYWNldGUQQWxpY2FudGUvQWxhY2FudAhBbG1lcsOtYQxBcmFiYS%2FDgWxhdmEIQXN0dXJpYXMGw4F2aWxhB0JhZGFqb3oOQmFsZWFycywgSWxsZXMJQmFyY2Vsb25hB0JpemthaWEGQnVyZ29zCEPDoWNlcmVzBkPDoWRpeglDYW50YWJyaWEUQ2FzdGVsbMOzbi9DYXN0ZWxsw7MFQ2V1dGELQ2l1ZGFkIFJlYWwIQ8OzcmRvYmEKQ29ydcOxYSwgQQZDdWVuY2EIR2lwdXprb2EGR2lyb25hB0dyYW5hZGELR3VhZGFsYWphcmEGSHVlbHZhBkh1ZXNjYQVKYcOpbgVMZcOzbgZMbGVpZGEETHVnbwZNYWRyaWQHTcOhbGFnYQdNZWxpbGxhBk11cmNpYQdOYXZhcnJhB091cmVuc2UIUGFsZW5jaWELUGFsbWFzLCBMYXMKUG9udGV2ZWRyYQlSaW9qYSwgTGEJU2FsYW1hbmNhFlNhbnRhIENydXogZGUgVGVuZXJpZmUHU2Vnb3ZpYQdTZXZpbGxhBVNvcmlhCVRhcnJhZ29uYQZUZXJ1ZWwGVG9sZWRvElZhbGVuY2lhL1ZhbMOpbmNpYQpWYWxsYWRvbGlkBlphbW9yYQhaYXJhZ296YRU1ATABMgEzATQBMQIzMwE1ATYBNwE4AjQ4ATkCMTACMTECMzkCMTICNTECMTMCMTQCMTUCMTYCMjACMTcCMTgCMTkCMjECMjICMjMCMjQCMjUCMjcCMjgCMjkCNTICMzACMzECMzICMzQCMzUCMzYCMjYCMzcCMzgCNDACNDECNDICNDMCNDQCNDUCNDYCNDcCNDkCNTAUKwM1Z2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2cWAQIBZAIbDxAPFgYfDQUFQ2xhdmUfDgUGTm9tYnJlHw9nZBAVWAVUb2RvcwlBYmVuZ2licmUGQWxhdG96CEFsYmFjZXRlCEFsYmF0YW5hB0FsYm9yZWEIQWxjYWRvem8SQWxjYWzDoSBkZWwgSsO6Y2FyB0FsY2FyYXoHQWxtYW5zYQZBbHBlcmEEQXluYQhCYWxhem90ZQ5CYWxsZXN0ZXJvLCBFbAxCYWxzYSBkZSBWZXMGQmFycmF4C0JpZW5zZXJ2aWRhB0JvZ2FycmEGQm9uZXRlC0JvbmlsbG8sIEVsCUNhcmNlbMOpbhVDYXNhcyBkZSBKdWFuIE7DusOxZXoQQ2FzYXMgZGUgTMOhemFybwxDYXNhcyBkZSBWZXMOQ2FzYXMtSWLDocOxZXoHQ2F1ZGV0ZQhDZW5pemF0ZRtDaGluY2hpbGxhIGRlIE1vbnRlLUFyYWfDs24MQ29ycmFsLVJ1YmlvCENvdGlsbGFzEkVsY2hlIGRlIGxhIFNpZXJyYQZGw6lyZXoJRnVlbnNhbnRhDUZ1ZW50ZS3DgWxhbW8NRnVlbnRlYWxiaWxsYQpHaW5ldGEsIExhCUdvbG9zYWx2bwdIZWxsw61uC0hlcnJlcmEsIExhCkhpZ3VlcnVlbGEMSG95YS1Hb256YWxvCEpvcnF1ZXJhBUxldHVyBkxlenV6YQdMacOpdG9yC01hZHJpZ3VlcmFzBk1haG9yYQhNYXNlZ29zbwZNaW5heWEJTW9saW5pY29zCU1vbnRhbHZvcxhNb250ZWFsZWdyZSBkZWwgQ2FzdGlsbG8JTW90aWxsZWphBk11bmVyYRFOYXZhcyBkZSBKb3JxdWVyYQZOZXJwaW8FT250dXIPT3NzYSBkZSBNb250aWVsElBhdGVybmEgZGVsIE1hZGVyYRNQZcOxYXMgZGUgU2FuIFBlZHJvClBlw7Fhc2Nvc2EIUMOpdHJvbGEJUG92ZWRpbGxhDFBvem8gQ2HDsWFkYQlQb3pvaG9uZG8MUG96by1Mb3JlbnRlB1BvenVlbG8LUmVjdWVqYSwgTGEHUmnDs3BhcgdSb2JsZWRvCFJvZGEsIExhB1NhbG9icmUJU2FuIFBlZHJvB1NvY292b3MVVGFyYXpvbmEgZGUgbGEgTWFuY2hhB1RvYmFycmEKVmFsZGVnYW5nYQZWaWFub3MMVmlsbGEgZGUgVmVzFlZpbGxhbGdvcmRvIGRlbCBKw7pjYXIKVmlsbGFtYWxlYQ1WaWxsYXBhbGFjaW9zDVZpbGxhcnJvYmxlZG8JVmlsbGF0b3lhDVZpbGxhdmFsaWVudGUYVmlsbGF2ZXJkZSBkZSBHdWFkYWxpbWFyB1ZpdmVyb3MFWWVzdGUVWAEwATEBMgEzATQBNQE2ATcBOAE5AjEwAjExAjEyAjE0AjEzAjE1AjE2AjE3AjE4AjE5AjIwAjIxAjIyAjIzAjI0AjI1AjI2AjI5AjI3AjI4AjMwAjMxAjMyAjMzAjM0AjM1AjM2AjM3AjM4AjM5AjQwAjQxAjQyAjQzAjQ0AjQ1AjQ2AjQ3AjQ4AjQ5AjUwAjUxAjUyAjUzAjU0AjU1AjU2AjU3AjU4AjYwAjU5AjYxAjYyAzkwMQI2MwI2NAI2NQI2NgI2NwI2OAI2OQI3MAI3MQI3MgI3MwI3NAI3NQI3NgI3NwI3OAI3OQI4MAI4MQI4MgI4MwI4NAI4NQI4NhQrA1hnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZGQCIw8QDxYGHw9nHw0FBUNsYXZlHw4FBk5vbWJyZWQQFQcFVG9kb3MJUHJvZHVjdG9yGEVsYWJvcmFkb3IvVHJhbnNmb3JtYWRvcgpJbXBvcnRhZG9yCkV4cG9ydGFkb3IPQ29tZXJjaWFsaXphZG9yG090cm9zIChhbG1hY2VuaXN0YXMsIG90cm9zKRUHATABMQEyATMBNAE1ATYUKwMHZ2dnZ2dnZxYBZmQCJw8QDxYGHw0FBUNsYXZlHw4FC0Rlc2NyaXBjaW9uHw9nZBAVJAVUb2RvcxFDVUxUSVZPUyBUT1RBTEVTIEhPdHJhcyBzdXBlcmZpY2llcyAoQ3VsdGl2b3MgZXNwZWPDrWZpY29zKSAoTm8gc2UgaW5jbHVpcsOhbiBlbiBlbCB0b3RhbCkHQm92aW5vcwdQb3JjaW5vBU92aW5vB0NhcHJpbm8OQXZlcyBkZSBjb3JyYWwIw4lxdWlkb3MLQ3VuaWN1bHR1cmEPT3RyYSBnYW5hZGVyw61hC0FwaWN1bHR1cmEgC0FDVUlDVUxUVVJBEkNhcm5lcyB5IGRlcml2YWRvcxZDb25kaW1lbnRvcyB5IGVzcGVjaWFzSkFsaW1lbnRvcyBlc3RpbXVsYW50ZXMsIGVzcGVjaWVzIHZlZ2V0YWxlcyBwYXJhIGluZnVzaW9uZXMgeSBzdXMgZGVyaXZhZG9zF0JlYmlkYXMgbm8gYWxjb2jDs2xpY2FzFEJlYmlkYXMgYWxjb2jDs2xpY2FzB1BpZW5zb3MvU2VtaWxsYXMgeSBtYXRlcmlhbCBkZSByZXByb2R1Y2Npw7NuIHZlZ2V0YXRpdm8rUGVzY2Fkb3MsIGNydXN0w6FjZW9zLCBtb2x1c2NvcyB5IGRlcml2YWRvcz5BbG1hY2VuYW1pZW50bywgbWFuaXB1bGFjacOzbiB5IGVudmFzYWRvIGRlIGdyYW5vcyBwYXJhIHBpZW5zbxJIdWV2b3MgeSBkZXJpdmFkb3MkUHJvZHVjdG9zIG9idGVuaWRvcyBkZSBsYSBhcGljdWx0dXJhEUxlY2hlIHkgZGVyaXZhZG9zMEFjZWl0ZXMgdmVnZXRhbGVzIHkgZ3Jhc2FzIHZlZ2V0YWxlcyBjb21lc3RpYmxlcx1DZXJlYWxlcywgaGFyaW5hcyB5IGRlcml2YWRvc4cBTWFuaXB1bGFjacOzbiwgdHJhbnNmb3JtYWNpw7NuIHkgZW52YXNhZG8gZGUgcHJvZHVjdG9zIGhvcnRvZnJ1dMOtY29sYXMgKGhvcnRhbGl6YXMsIHR1YsOpcmN1bG9zLCBsZWd1bWJyZXMsIGZydXRhcywgc2V0YXMgeSBkZXJpdmFkb3MpIEF6w7pjYXJlcywgZGVyaXZhZG9zIHkgc2ltaWxhcmVzK0NvbWlkYXMgcHJlcGFyYWRhcyB5IGFsaW1lbnRhY2nDs24gZXNwZWNpYWwHSGVsYWRvczdBbG1hY2VuYW1pZW50byB5IGNvbnNlcnZhY2nDs24gZGUgcHJvZHVjdG9zIGVjb2zDs2dpY29zFVByb2R1Y3RvcyBlY29sw7NnaWNvc0ZBbG1hY2VuYW1pZW50bywgbWFuaXB1bGFjacOzbiB5IGVudmFzYWRvIGRlIGdyYW5vcyBwYXJhIGNvbnN1bW8gaHVtYW5vNk90cm9zIHByb2R1Y3RvcyBkZSBsYSBpbmR1c3RyaWEgYWdyb2FsaW1lbnRhcmlhIG4uYy5vLhNBQ1VJQ1VMVFVSQSAoQUxHQVMpFSQBMAI1OAMxMTcDMzIyAzMyNQMzMjcDMzI5AzMzMQMzMzcDMzM4AzMzOQMzODgDNDAwAzk0NAM5NDYDOTQ3Azk0OAM5NDkDOTUxAzk1NAM5NTYDOTU4Azk1OQM5NjADOTYyAzk2NQM5NjYDOTcxAzk3MgM5ODUDOTg2BDEwMDIEMTAwMwQxMDEyBDEwNjQEMTA2NRQrAyRnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2cWAWZkAisPEGRkFgBkAi8PEGRkFgBkAjMPEGRkFgBkAjcPEGRkFgBkAjsPEA8WBh8NBQVDbGF2ZR8OBQZOb21icmUfD2dkEBUUBVRvZGFzCkFuZGFsdWPDrWEHQXJhZ8OzbhdBc3R1cmlhcywgUHJpbmNpcGFkbyBkZQ5CYWxlYXJzLCBJbGxlcwhDYW5hcmlhcwlDYW50YWJyaWEUQ2FzdGlsbGEgLSBMYSBNYW5jaGEQQ2FzdGlsbGEgeSBMZcOzbglDYXRhbHXDsWEFQ2V1dGEUQ29tdW5pdGF0IFZhbGVuY2lhbmELRXh0cmVtYWR1cmEHR2FsaWNpYRRNYWRyaWQsIENvbXVuaWRhZCBkZQdNZWxpbGxhEk11cmNpYSwgUmVnacOzbiBkZRtOYXZhcnJhLCBDb211bmlkYWQgRm9yYWwgZGULUGHDrXMgVmFzY28JUmlvamEsIExhFRQBMAExATIBMwE0ATUBNgE4ATcBOQIxOAIxMAIxMQIxMgIxMwIxOQIxNAIxNQIxNgIxNxQrAxRnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZxYBZmQCPw8QDxYGHw0FBUNsYXZlHw4FBk5vbWJyZR8PZ2QQFTUFVG9kYXMIQWxiYWNldGUQQWxpY2FudGUvQWxhY2FudAhBbG1lcsOtYQxBcmFiYS%2FDgWxhdmEIQXN0dXJpYXMGw4F2aWxhB0JhZGFqb3oOQmFsZWFycywgSWxsZXMJQmFyY2Vsb25hB0JpemthaWEGQnVyZ29zCEPDoWNlcmVzBkPDoWRpeglDYW50YWJyaWEUQ2FzdGVsbMOzbi9DYXN0ZWxsw7MFQ2V1dGELQ2l1ZGFkIFJlYWwIQ8OzcmRvYmEKQ29ydcOxYSwgQQZDdWVuY2EIR2lwdXprb2EGR2lyb25hB0dyYW5hZGELR3VhZGFsYWphcmEGSHVlbHZhBkh1ZXNjYQVKYcOpbgVMZcOzbgZMbGVpZGEETHVnbwZNYWRyaWQHTcOhbGFnYQdNZWxpbGxhBk11cmNpYQdOYXZhcnJhB091cmVuc2UIUGFsZW5jaWELUGFsbWFzLCBMYXMKUG9udGV2ZWRyYQlSaW9qYSwgTGEJU2FsYW1hbmNhFlNhbnRhIENydXogZGUgVGVuZXJpZmUHU2Vnb3ZpYQdTZXZpbGxhBVNvcmlhCVRhcnJhZ29uYQZUZXJ1ZWwGVG9sZWRvElZhbGVuY2lhL1ZhbMOpbmNpYQpWYWxsYWRvbGlkBlphbW9yYQhaYXJhZ296YRU1ATABMgEzATQBMQIzMwE1ATYBNwE4AjQ4ATkCMTACMTECMzkCMTICNTECMTMCMTQCMTUCMTYCMjACMTcCMTgCMTkCMjECMjICMjMCMjQCMjUCMjcCMjgCMjkCNTICMzACMzECMzICMzQCMzUCMzYCMjYCMzcCMzgCNDACNDECNDICNDMCNDQCNDUCNDYCNDcCNDkCNTAUKwM1Z2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2cWAWZkAgMPZBYCAgEPPCsAEQIBEBYAFgAWAAwUKwAAZAIFDw8WAh8JZWRkGAIFHl9fQ29udHJvbHNSZXF1aXJlUG9zdEJhY2tLZXlfXxYBBStjdGwwMCRDcGhDdWVycG9QYWdpbmEkY2hrU2VkZVNvY2lhbE90cm9QYWlzBSRjdGwwMCRDcGhDdWVycG9QYWdpbmEkR1ZDZXJ0aWZpY2Fkb3MPZ2QDChNGIM1OX10%2BeyZHeUfX2e8H7qcrnW2VdnAr5Bs9ZA%3D%3D&__VIEWSTATEGENERATOR=803872E2&__EVENTVALIDATION=%2FwEdAKICQ%2FsatO%2FdZYGuGuinR1U2gyVL6hjUg98iGQVswT08Zk5q0PBQolou6mbgolXi%2BJqWEaV7%2FUljejfIxzRhFPN55bpUkxedWxIvhPaGwP3Armkl8rgHQDii%2BpFMl3v6MLrrDcHYY3OJVTK9XrcpeJZRXx3oPFJbFwp%2FoKMcaM6fgpYD4mdIj4YMCbWd%2BlHUd%2BqQo%2B%2FdIEak%2FwpDSFcImrFMZGiMQBpRCO02ED%2B219Iwjtu5X8fEEgPW1bDnvqPR8VbH5bxr2B5l4cqyNEMsm%2F119jP1TcZXMWwoOOWsxneBAL3t1mb8SfLgbgZTVewYLdqpsm85deCwjWVlpUzSE%2B7xOmym8Dqluv2EldiT9uCZ73LLn7yeND0WTkRkelrM9VZqULo48WdoGD5d83leXnGBFK6JoerVvbjv5GdEZS1np%2Bd0goc2yVmLIe7c%2BXUE%2FEhg4O%2Fq5zT57Jlk4HONlhUWmTS2Sj%2FOB6shiH2%2FodenSYehKjL3J1wniUr6a9whhLZlC5lvHgZPHGsVp4kZozlCqneSSYruU%2F37TG2P2ZDc1xvReaMfjWOXuumh2a2k%2BWqkWlYHy0VzJ%2B3IUeZlznpZmsmGAeY3nxRMbwOXIF1RgILfCh661qOuS%2BwIvTPDFiXhWp88%2BQt5QrC0VdShd%2FMewJU8M4tDfISiTZxhjB2jDE%2BULbMejGMRNOwcfd%2B8Hgx0zOlWXYxxU5bDrXJfZWu9DF14RK3po4MXFGzyrSE9gmSrOkoPReFeYn9XlQz%2BFyblVXVrRV4hUkTbA0i%2BjP35W1Xpemucv8kAZhz24uTekVHhjg4CZ9QEOaVjL5t43Tt0hg6u87oyxjGar2ya7mpbrQpqXizCuCBLMX3OT5kfjc0KoZTisHSYgCTTPOo6MFcaIQWEmrlHJ71Qa2Ie8xBFNz%2FIVTKsTW8NeuqEXJNgcdfRAlJWBUdvVRy3UZ22DWk4PYhB92INR8tbIgcqdbkmVQoqNVmQYWdivNG3lZQJx57vlUiyZpA8l21rlumDyyjVAvyeAJSrt%2BvAmxxKHZ%2B8FA8Biojf05FkswhkOMQggBkSIpCzW86%2FQ0%2BeHS3f62V7%2FrsQj9nWuaseYkXLXiPJLKky%2FOanlK03nUSa7aZXxeONOrLtVKVGLyEWEsgzrz3XICKCZb2%2ByVaWJQj%2FsTMnxA2r%2FWM02btIpGWLb3qDWCPrLGwsRnvyuF4uS1RiFki%2Brv7XbDMQt5t4wDeW71OVY%2FjCMs7%2B0J%2FAF8lop1BKFIcJ%2B6AGqixy7FDhzYWMI05rOTIs5Mgl5hq%2FOernl4airGj3XMc9sdQzrkj35Jw0BJaYaeqMxC3%2FmxkDZrFZCTz1ZDqc37ARA6gkPAdNPVvjguEaBgBejOlh1O2sDhKvNtOWr0GyGlfJkcNK3IC0jZnJopIO2gl6T1eOBrG9du7bg%2B9C5eQ%2FTVEuf4Q95s9fZ55d5l3oV2WkssVjBZ6AvY6q0FTMfJ00vS9KiArzJQRmle88tOHiHTbORNzZuEc4dEbURGBPY9fZVfPLjy6IZSAulOQTJXi%2B7OrU0yZQuNlIvNz84v46uguPPKb1YhbGDWj5Tz0MNYSxJZyvNBCr%2FAaU%2BS1%2BiauQd0BiF1OK3hVSKJSKnINAAdlsvhPmd2zMaht7KAs1IWmRtKIklsAS98hclGVTLtsOcaYRkfn1yzkuqwNkFeH6IjWjgOh828JpzIrpzduvnbJ47kOFWJ2Bd4bHuzCqgFP4xp03kM0RyoUPO83EhX0A%2B0Vt8311nbKOiqGKVWrOeHf8a%2Fn7x0KAsvyLUFzt8US5oSaoc9T9yItozmVP%2Fn%2Fljx3nRWtwWmAhRne0kRWDxI%2FTc%2FVUTww92x7GOm%2FfWj%2FVY2rSY9N55qjL1SutFaADJL5Mlkmc5Y%2BJZ%2Frkv7LRtyuexowPpGM3Kujef1nWcdHMGoiQxpATDcd%2FZgYHDiQXfso%2F%2BcfIKiAiV%2F5Og55bf8ulxk22y0lke%2F8kl95991Bd47woG9vIjsXEkf4i4ajKUlCtmf7PfjKacJZ4Lz0iC0UTWKLm%2F64r2ZAuSR9AkK30D8lHJKoOAbO1eJJP3BJK0yqbcprINEgcxkQlO8VcoD620BGFUBZnxgy5boKhask4L2FaL1MmA93erDQr06u08mukSgKMQrAriYeWVWKfDgA0dzF%2Bvr4sRaKEOU9Dvcox9OigFySnLEa47XFvT3960mgKVOkUpGaJKY2e7hiI%2Fbjw4ChMAfeAuEF2mLFQ806x55xCZJBYPoIVFkN0jE3kfd7LTzV8czEomfqXjxCGewJDvErqcnwiPiF6YH9VdwAxed%2Fb%2Fg6DngwsaNP9jXvow5N9aQOE1xR2W0B7UIuFWN7tAJRj3mDzX6fvHoCiDvF7vo4DRNydjCXGDwM3aQYkqC3jgZpVlPItzq%2Ff54LJW9GmDmO5qshwoJVvvkaV7exkka4SoyW%2BSrZV8asTZ59sfKt4gt35X%2BovJ9h3%2FyeF5%2F1drXdZxluwfQxHcoW5w%2BjAXaeK28lbY%2FyiPh9s9Di2cXj8L1luV1wpMIBRSE2TcJTvlPkerD3lBDzGCY0BKEKZ4q8IaM6X1wA9lQwW8KY22IQFw%2B8CWL%2FV5BGvzsjgEkM7bkCj86hArEsXArZruiuLYKOqNrt%2F1b8to7I1zv4OgC%2B0ABnBO0AMgDRmcfc4GqEnZbzxfwKBEbS6TV6Kg3nL%2B5ZoJrdzK64%2BcpfJgx1uEtBRBiyKuVBLevGbvtAOYAWJih6iLqdbqcM%2FHAapr264PYFlbBKAJON05gxsmrVR8EXbIiK8EeHdcWl6KbsutOvx%2FebHpph%2BCHUutvT0XHH%2FGxgW4eG7w2AQeqlheL6LcjQ4LEM1s6Eovgijf7vlyG5F4e4t%2FbPtL3XKM0Lb%2FrwxZD37kvIBoB3krwi1BqnTCPPU16TKS%2BEkY4weypIjOvoFWGa9jbyn5KEQYObeA4fjzsbOxBoOWKoVdpkRnkkJDCrkygek6LQ%2Fv6B08N8qJdKsafugrWGWdzL63esXndz6HBRVumyvtjCz1N%2BRA7%2BpJeRCpBwyECFsXtz2vN0CW4V2kpVZG6k2uEoe3X8HqDHo90z%2Fv5Vb33dOA%2BL%2BsZvfaBx3ScWybNOPKr8tv6asdL5oJ9QkjCehdJNlD%2FBsUhK%2BlYPc8KbyDqhPzKJ6d3Ohce11q1I25%2FSry2pGvFbumK2y0RYZRZh6Owj3HvBGQJ8SN49U%2Bwa2KblWwpbUKzkeQM3wBWL3Bt7RxgadCtMRJ60YtnP0u2nRVwZ635q5Jki7oJNNl8LSHf5OuvW82lL%2BnfNNA7CtrYfkGPYl4YeWCCa4T7F57sUwm7Qoy2Kf1H2fRxw346io%2B60OYQfNkyVD3uNkxyQnTaUNwFakFkwrzIGLO%2BNyAo8U7KMfBJk5vTKj%2Fy2o4ax3JZG%2B6ffCCdQQ6CYtFB2jrnNRPCpcA9BYeBQNfx7H1HlmUGSdiZx1V8Oge%2FXP%2FNkIIVJvlg8g7ZN0YoVokJuv9onUiun7fqsi8lgALNTZY0PWeaJbqjyWin%2BBbLSYKx4Zm%2B4AUlJiwDM%2FzRSIq7rIlC0UZRnA1TEXUhvRa9nJn9PNu4SfB5bwAg2aTCYFpEGEM3JgqBALkmELUcMRvIdi8CUdJukESPW70cH2%2BexDh%2FHbIMI2vm8NLfo6M%2F1YJPe3O6QyntTrBOcohHDSaglahjTMArtKXgXQAc6z3uPuEo17iMya4UIAxILbbTGKVHuMTvGKqgXlCybJV87jft3QC6yzXxUKKkj%2FxoYqJXm6hHS1OWkPqw5wjrfi%2B2DWLb4dVfEbtn8YMu5D9cYL3Yf10m%2BbI4EC8eMFM1HSnTmJFl5ZX%2FnTnKmWyGwDHO3bb2uDir4UgCdVxujpdRg7kbxQLxx69chMvD9FV0AKiE1y25PfbaueF2gO77LXvcQxZbxS4L5m4D5P64P9rrjoQAjfJ7PmvVIwgJHwRvsICiCHqRHX3xWRw3uNFm9orxzZfYxjFxpxRwCE%2BEpRBKOX5ryXH1EJ0W1s8pUglu5MXqzOcdJKigjKe3iIVCx1owLQ9BqaWFiRd07kb51AmIeY8ndL9nt7tYsi0NDkaLVd%2B2UG4oWE49t%2FaWgMMnLSHfn1DC6aM9iKbwCT6LfdFEjy6gHXT%2BL8lWpYHCptyf%2FfOQPjrDNIXO%2BPMZjH0pecoRbP3zVyrlQTSpDdclyOm8vVQNAJ670dKn%2FDN1%2BZ5WMDmZ5eCanjqZTHcyCC%2BhXb7GU8VluipiCvhoiaQHcTd%2Ble4yu%2BfqKx27wJ16Ffp0pysF%2B5YQxdRm75D5mGtOGtNJx6eYYG6PwJHfLPZKscuK9HFsGzNgQP0Skjmnlz2IDBnhYbRoh1ThRk18sVCuipDu8nTY3dZj%2B%2BCX2Vf1yz12VnP8iMJr3VwDt8ZYrNMpjlE9ha63mh8OotKteycRLFcqyxXX6kNbDJl4QHAkHH3WrRNwj1Z1tJapuRjFjsMUQNlD4mIRcYkklUQuZo0AA1pDELzUWkLYzwyiWU2ACQ1DnMt005BrWwVjXH69m3nM%2FU0hX6qXNTOzaNFx8feHuSHoh5w3P6yDjEWI%2B8DZdELk1MUSQiju1yzliuUbipGZVbqHpavH6XwrlQ8v4PlW5W6kcsVMuMjFwNy6u6x%2FE6QHCdxC8eIgzj3v3czUCHa9oVzXefbHnpgDzTPNMP2JgoT1O8QqAkTZr6K5Kyq6FUrUyCodx6fzK%2FpXeQ7vZfDnyS%2FLu8BlwdDy6lDZuZJbqayj%2FwNQBM0h2rXXSj%2BBfCgsxzV6n7o7mkgL1OcuZpnSME2OArHN4RokCUxE9J8vWWHdMFdsUKJEBTvn6SXGBmUqIrdHTfZ42%2BKq%2FaijZWW5kRR0EfVFgjjJGzuMIX7qyY91DdSiR8rM1TCILpgZ4NB1%2FuSk%2FbGP%2FXavuSXerQ0%2BbSIxRwPbneFiVk5MBenyj3ZQUGLIRzVzbeh6L4Kh5wPL52j%2FXfEYfKsJCF0hzFFLCgZNlfKNWQ3Y4VjxO0ep4narb5m0mxcUnPMzgC6gOFMNLhAopZviSt6DJNDvaKHnF50Go8BqyMBL3PDv3qdRD262gxUrOsHYrtsppnvbzrh66n4ESyLquRuTwLO%2FzT3d9mh0w%2FT0hWuxiqwMbyn4rxUcYCIggGnArRc8quHCjLnivpaJGBVeWhV06GBiiNJGtYC%2BHE%2FR19TSgLnGLDqWuwrVCL6uLHd2nH1kYwYPfncx4NqP7wMW8yFp7zDWykIVZNQaUUmjuFTpSTDHU9KyRZjkW9wYpvVUyKM6s%2FJpY6jzq6GBIw4tvf5cZymKzwgEEnM%2FKtTcxyFw0j4JSHsJ7uCdH9gjcFOOTbLw05qqzx1LgPkP5WbkOTk0jgNjuh7dbFNPQvkfl0pOIgyDlk2JIrIpH%2Fd7hti7HgU8NGqxNUEHTlAXeBu05%2FGppBmZRUfFt3x6qcphkQvZ5hUocmT5RJvCuNcf2AQad9IzjRD%2B89PIgkaZsB6R%2Fb%2B1OZVyGLcToYIdOcZO7MT5XWveywIvWB5mru9EOrfk%2FKe17y55h%2F5t3kqqhpT1DAq%2Bc%2F0sxo%2FGiv0ACEI3WG8hnqHACWixigMl0w5i%2Bz7WoP5y62sxmliBxy31tm7hs8kWPLC8SwLG1lYJ29dSiALO7gP2hAMC73giSohx5FWaw5%2BhbWYYkyxNYBZHlWYXadOgWWMfJkZOoQ84nhsIVS6ca9UoQ%2F2jTXKd9YJi%2B0XYZYp8fv0btZ0fq8bVaJQm511U7o1tMJVhsVP2QTNJl9jqh9%2FzfQtMhTRl2ehsyApMHfYKxXMFfWJIhPXsweM%2Fgnlg6%2BiPaLv7q2wLCYazCHr7XgLx1GA6NIjah3NVhDoYfVldmv0dLau9hLe0gdudy4CQUCFVmR95dgJFlC2WWCNePvFMCxC%2Bo98CbfsIIlXaXOcFMSvB86w9s6QkAZL%2FLC%2BKKOkO%2BUTRv3xXIo%2Bw9BX85iAfmmbvovsPyGDeizNYIcA2BeqBFeK49Kt%2B78HRyXv8McaCcJ%2BrqJ8NpRjnMg%2FECweTzxaw7nw72j0o7UszDP7bR8rymst6hk%2FRbXoe0O2QH560slr8WBjoyWdM8Q9uyjmfbFCyANSJ7%2FpZNl5v4eaorICbIikgTxW9jDfQmKkMppUT%2Bs1rcysuApnshjZ3tNKLJz1b0YLTPz%2FFYNd22Da%2FD1I4MTdeEJsuxrO2CgymiQyO8f6uqMPDBqA0ov4eCVLQA%2BZloXQjDvU%2FkEaZ0FIg%3D%3D&ctl00%24
    ctl00$CphCuerpoPagina%24
    txtNombre=&ctl00%24
    CphCuerpoPagina%24
    txtDireccion=&ctl00%24
    CphCuerpoPagina%24
    txtCodigoPostal=&ctl00%24
    CphCuerpoPagina%24
    ddlCCAA=0&ctl00%24
    CphCuerpoPagina%24
    ddlProvincias=2&
    ctl00%24
    CphCuerpoPagina%24
    ddlMunicipio=0&ctl00%24CphCuerpoPagina%24txtActividad=&ctl00%24CphCuerpoPagina%24ddlSubTipoOperador=0&ctl00%24CphCuerpoPagina%24ddlActividadNivel1=0&ctl00%24CphCuerpoPagina%24ddlCCAAUbicacion=0&ctl00%24CphCuerpoPagina
    ddlProvinciasUbicacion=0&ctl00
    CphCuerpoPagina
    btnBuscar=Buscar
}

In [62]:
params = {

'__LASTFOCUS':"",
'__EVENTTARGET':"ctl00$CphCuerpoPagina$GVCertificados", # "ctl00$CphCuerpoPagina$ddlProvincias",
'__EVENTARGUMENT':"Page$2",
#"__VIEWSTATE":"...",
#"__VIEWSTATEGENERATOR":"803872E2",
#"__EVENTVALIDATION":"/wEdAP4EpBkGQ+20Onjt+UaH3zzssCVL6hjUg98iGQVswT08Zk5q0PBQolou6mbgolXi+JqWEaV7/UljejfIxzRhFPN55bpUkxedWxIvhPaGwP3Armkl8rgHQDii+pFMl3v6MLrrDcHYY3OJVTK9XrcpeJZRXx3oPFJbFwp/oKMcaM6fgpYD4mdIj4YMCbWd+lHUd+qQo+/dIEak/wpDSFcImrFMZGiMQBpRCO02ED+219Iwjtu5X8fEEgPW1bDnvqPR8VbH5bxr2B5l4cqyNEMsm/119jP1TcZXMWwoOOWsxneBAL3t1mb8SfLgbgZTVewYLdqpsm85deCwjWVlpUzSE+7xOmym8Dqluv2EldiT9uCZ73LLn7yeND0WTkRkelrM9VZqULo48WdoGD5d83leXnGBFK6JoerVvbjv5GdEZS1np+d0goc2yVmLIe7c+XUE/Ehg4O/q5zT57Jlk4HONlhUWmTS2Sj/OB6shiH2/odenSYehKjL3J1wniUr6a9whhLZlC5lvHgZPHGsVp4kZozlCqneSSYruU/37TG2P2ZDc1xvReaMfjWOXuumh2a2k+WqkWlYHy0VzJ+3IUeZlznpZmlqfPPkLeUKwtFXUoXfzHsAKal4swrggSzF9zk+ZH43NPYhB92INR8tbIgcqdbkmVRIikLNbzr9DT54dLd/rZXupMvzmp5StN51Emu2mV8XjICKCZb2+yVaWJQj/sTMnxAn7oAaqLHLsUOHNhYwjTmvURGBPY9fZVfPLjy6IZSAu6c3br52yeO5DhVidgXeGx7swqoBT+MadN5DNEcqFDzvNxIV9APtFbfN9dZ2yjoqhilVqznh3/Gv5+8dCgLL8i1Bc7fFEuaEmqHPU/ciLaM5lT/5/5Y8d50VrcFpgIUZ3tJEVg8SP03P1VE8MPdsexjpv31o/1WNq0mPTeeaoy9UrrRWgAyS+TJZJnOWPiWf65L+y0bcrnsaMD6RjNyro3n9Z1nHRzBqIkMaQEw3Hf2YGBw4kF37KP/nHyCogIlf+ToOeW3/LpcZNtstJZHv/JP4i4ajKUlCtmf7PfjKacJaX3n33UF3jvCgb28iOxcSReC89IgtFE1ii5v+uK9mQLkkfQJCt9A/JRySqDgGztXiST9wSStMqm3KayDRIHMZEJTvFXKA+ttARhVAWZ8YMuW6CoWrJOC9hWi9TJgPd3qzFdInlCxVyqktvWeSgX56/NCvTq7Tya6RKAoxCsCuJh5ZVYp8OADR3MX6+vixFooQ5T0O9yjH06KAXJKcsRrjtcW9Pf3rSaApU6RSkZokpjZ7uGIj9uPDgKEwB94C4QXZDdIxN5H3ey081fHMxKJn6YH9VdwAxed/b/g6DngwsaNP9jXvow5N9aQOE1xR2W0CXjxCGewJDvErqcnwiPiF6e1CLhVje7QCUY95g81+n7x6Aog7xe76OA0TcnYwlxg8DN2kGJKgt44GaVZTyLc6v3+eCyVvRpg5juarIcKCVb3f/J4Xn/V2td1nGW7B9DEdyhbnD6MBdp4rbyVtj/KI+vkaV7exkka4SoyW+SrZV8asTZ59sfKt4gt35X+ovJ9gfbPQ4tnF4/C9ZbldcKTCAd5WiaTEuhBsW2S0ipu68DQwW8KY22IQFw+8CWL/V5BECtmu6K4tgo6o2u3/Vvy2jsjXO/g6AL7QAGcE7QAyANGZx9zgaoSdlvPF/AoERtLpNXoqDecv7lmgmt3Mrrj5yl8mDHW4S0FEGLIq5UEt68Zu+0A5gBYmKHqIup1upwz8cBqmvbrg9gWVsEoAk43TmDGyatVHwRdsiIrwR4d1xaXopuy606/H95semmH4IdS629PRccf8bGBbh4bvDYBB6qWF4votyNDgsQzWzoSi+CKN/u+XIbkXh7i39s+0vdcozQtv+vDFkPfuS8gGgHeSvCLUGqdMI89TXpMpL4SRjjB7KkiM6+gVYZr2NvKfkoRBg5t4Dh+POxs7EGg5YqhV2dPDfKiXSrGn7oK1hlncy+pkRnkkJDCrkygek6LQ/v6Dd6xed3PocFFW6bK+2MLPU35EDv6kl5EKkHDIQIWxe3N1/B6gx6PdM/7+VW993TgPi/rGb32gcd0nFsmzTjyq/yojl3HSdjeSf+L/cVX41KC2/pqx0vmgn1CSMJ6F0k2UP8GxSEr6Vg9zwpvIOqE/Monp3c6Fx7XWrUjbn9KvLaka8Vu6YrbLRFhlFmHo7CPce8EZAnxI3j1T7BrYpuVbCltQrOR5AzfAFYvcG3tHGBp0K0xEnrRi2c/S7adFXBnrfmrkmSLugk02XwtId/k669bzaUv6d800DsK2th+QY9iXhh5YIJrhPsXnuxTCbtCjLYp/UfZ9HHDfjqKj7rQ5hB82TJUPe42THJCdNpQ3AVqQWTCvMgYs743ICjxTsox8EmTm9MqP/LajhrHclkb7p98IJ1BDoJi0UHaOuc1E8KlwD0Fh4FA1/HsfUeWZQZJ2JnHVXw6B79c/82QghUm+WDyDtk3RihWiQm6/2idSK6ft+qyLyWAAs1NljQ9Z5oluqPJaKf4FstJgrHhmb7gBSUmLAMz/NFIirusiULRRlGcDVMRdSG9Fr2cmf0827hJ/DfUiN8yyuav3gw02W1r1gsL/adCjVyK7LxthfGVfDiOzGGFmxbFdMPv7lSn4UTGU634Wo3pVN8H/V+dpCMORmsXQ34u3QQuGM6xM0xfU9qXdIaWWtgPG+3kYmCO0tcYj2vN0CW4V2kpVZG6k2uEoeo+a0svfPjiuBjiUcLlr+MkVibh07RbrEbkXP20fx/tNZLAF9RAz+C7m1bLs3lC9i2JyRh+Cm7o6LsB8XiRx9y87PyFTnG3O44uUmMAaMDgSvMeVYldvLkhAB/yoI4vG4tKV8o0BtzIYqrko/SKrfb4GCBZErrxAQAUs3gPZ848utozgZP6QktXph3kFESTvU3hwfEtTcI9YMcw+Sek6B6LETM9yoBY0AYLc7eWWPDQoHlvACDZpMJgWkQYQzcmCoEAuSYQtRwxG8h2LwJR0m6QRI9bvRwfb57EOH8dsgwja+bw0t+joz/Vgk97c7pDKe1OsE5yiEcNJqCVqGNMwCu0peBdABzrPe4+4SjXuIzJrhQgDEgtttMYpUe4xO8YqqBeULJslXzuN+3dALrLNfFQoqSP/GhiolebqEdLU5aQ+rDnCOt+L7YNYtvh1V8Ru2fxgy7kP1xgvdh/XSb5sjgQLx4wUzUdKdOYkWXllf+dOcqZbIbAMc7dtva4OKvhSAJ1XG6Ol1GDuRvFAvHHr1yEy8P0VXQAqITXLbk99tq54XaA7vste9xDFlvFLgvmbgPk/rg/2uuOhACN8ns+a9UjCAkfBG+wgKIIepEdffFZHDe40Wb2ivHNl9jGMXGnFHAIT4SlEEo5fmvJcfUQnRbWzylSCW7kxerM5x0kqKCMp7eIhULHWjAtD0GppYWJF3TuRvnUCYh5jyd0v2e3u1iyLQ0ORotV37ZQbihYTj239paAwyctId+fUMLpoz2IpvAJPot90USPLqAddP4vyValgcKm3J/985A+OsM0hc748xmMfSl5yhFs/fNXKuVBNKkN1yXI6by9VA0AnrvR0qf8M3X5nlYwOZnl4JqeOplMdzIIL6FdvsZTxWW6KmIK+GiJpAdxN36V7jK75+orHbvAnXoV+nSnKwX7lhDF1GbvkPmYa04a00nHp5hgbo/Akd8s9kqxy4r0cWwbM2BA/RKSOaeXPYgMGeFhtGiHVOFGTXyxUK6KkO7ydNjd1mP74JfZV/XLPXZWc/yIwmvdXAO3xlis0ymOUT2FrreaHw6i0q17JxEsVyrLFdfqQ1sMmXhAcCQcfdatE3CPVnW0lqm5GMWOwxRA2UPiYhFxiSSVRC5mjQADWkMQvNRaQtjPDKJZTYAJDUOcy3TTkGtbBWNcfr2becz9TSFfqpc1M7No0XHx94e5IeiHnDc/rIOMRYj7wNl0QuTUxRJCKO7XLOWK5RuKkZlVuoelq8fpfCuVDy/g+VblbqRyxUy4yMXA3Lq7rH8TpAcJ3ELx4iDOPe/dzNQIdr2hXNd59seemAPNM80w/YmChPU7xCoCRNmvorkrKroVStTIKh3Hp/Mr+ld5Du9l8OfJL8u7wGXB0PLqUNm5kluprKP/A1AEzSHatddKP4F8KCzHNXqfujuaSAvU5y5mmdIwTY4Csc3hGiQJTET0ny9ZYd0wV2xQokQFO+fpJcYGZSoit0dN9njb4qr9qKNlZbmRFHQR9UWCOMkbO4whfurJj3UN1KJHyszVMIgumBng0HX+5KT9sY/9dq+5Jd6tDT5tIjFHA9ud4WJWTkwF6fKPdlBQYshHNXNt6HovgqHnA8vnaP9d8Rh8qwkIXSHMUUsKBk2V8o1ZDdjhWPE7R6nidqtvmbSbFxSc8zOALqA4Uw0uECilm+JK3oMk0O9ooecXnQajwGrIwEvc8O/ep1EPbraDFSs6wdiu2ymme9vOuHrqfgRLIuq5G5PAs7/NPd32aHTD9PSFa7GKrAxvKfivFRxgIiCAacCtFzyq4cKMueK+lokYFV5aFXToYGKI0ka1gL4cT9HX1NKAucYsOpa7CtUIvq4sd3acfWRjBg9+dzHg2o/vAxbzIWnvMNbKQhVk1BpRSaO4VOlJMMdT0rJFmORb3Bim9VTIozqz8mljqPOroYEjDi29/lxnKYrPCAQScz8q1NzHIXDSPglIewnu4J0f2CNwU45NsvDTmqrPHUuA+Q/lZuQ5OTSOA2O6Ht1sU09C+R+XSk4iDIOWTYkisikf93uG2LseBTw0arE1QQdOUBd4G7Tn8amkGZlFR8W3fHqpymGRC9nmFShyZPlEm8K41x/YBBp30jONEP7z08iCRpmwHpH9v7U5lXIYtxOhgh05xk7sxPlda97LAi9YHmau70Q6t+T8p7XvLnmH/m3eSqqGlPUMCr5z/SzGj8aK/QAIQjdYbyGeocAJaLGKAyXTDmL7Ptag/nLrazGaWIHHLfW2buGzyRY8sLxLAsbWVgnb11KIAs7uA/aEAwLveCJKiHHkVZrDn6FtZhiTLE1gFkeVZhdp06BZYx8mRk6hDzieGwhVLpxr1ShD/aNNcp31gmL7Rdhlinx+/Ru1nR+rxtVolCbnXVTujW0wlWGxU/ZBM0mX2OqH3/N9C0yFNGXZ6GzICkwd9grFcwV9YkiE9ezB4z+CeWDr6I9ou/urbAsJhrMIevteAvHUYDo0iNqHc1WEOhh9WV2a/R0tq72Et7SB253LgJBQIVWZH3l2AkWULZZYI14+8UwLEL6j3wJt+wgiVdpc5wUxK8HzrD2zpCQBkv8sL4oo6Q75RNG/fFcij7D0FfzmIB+aZu+i+w/IYN6LM1ghwDYF6oEV4rj0q37vwdHJe/wxxoJwn6uonw2lGOcyD8QLB5PPFrDufDvaPSjtSzMM/ttHyvKay3qGT9Fteh7Q7ZAfnrSyWvxYGOjJZ0zxD27KOZ9sULIA1Inv+lk2Xm/h5qisgJsiKSBPFb2MN9CYqQymlRP6zWtzKy4CmeyGNne00osnPVvRgtM/P8Vg13bYNr/s5BVzyHJSHKOzA/MAjH05Y7GH7fEzqVhicduShTWZKaeSXBrZXZyPjFG5SGD0x6JN9VC5NvrSjsDvtbSwqXPuyyfFXnDNxFwuvRdunIPhLSp2XmC515SCmQfRmb6xac3BRgvEEKaR81eRNsacB8PJTK8yBe+FGwWTBStF4EmxrQ6aP6qH5kTbKsPp1WqiNTnMZXEk5QJlJfo5Y8/30X7LukQUcOhMaWc1VOZ4cxRB+QlZFhX0rdmNdsOyCmvl60v/6jC4mkbIsGNoi1QXNsuBPWvv0O+0ZxxXh4SEi9k/+gl4KSr23BdzMVFK1JFbMPgeLkNFIamYEVeR/7uMlvHgaW8uLk8gCI0znOcSijFdvxxaxfGS0ZwACW1d0u1VcHbdL106F+4rHwvPDZ7YV4uIaI3ICxwoVxQwE59KgzZ8p7ooEvTR/qxONyEdgx0JjDQisvQdlfNzlxbEhZu4CpFHN1n8l4pkVnYY34fyQJk+BHwXUMse4SF2t6d7Cnhn9loU8JFrk1eoOAzQl8Vna1WDjvL1eXeRr4Pgdx4hM79/cOxCbKpenca6Q12rfbAgdKFSQUSNLeuWAxqwjwBEfIc/5qzuqcXhBv5S6C73/k5pkaqC5B5xg0jgkecc/XYuXVKo38nIJyD1NV+Q0wgPRpbF188hiECMrIBtIXEDxKkggCEIcw/Zrfd9Xuq0TdmnAOMSpZSZBX7Lm7n5dPmrRLIwatBIyD8Dx1lQFBAEtAyGkvBnXFs9rrpagkP8WFE3q+ykqj2A9pkTeFTOZ1mXdxBFubduElg0Iwmuwc1pBgf8sGY/7DueJlauInXQ7dc7paOpXa80pW0J1npgznICi2B+Ypll/6HUaYZYZwrHk0F7cMMhO9UhnTxyGOj/SR6r/FsqxWsP28vqUID0eupN0nMytSbZVf3OgGWgLkOy5lziyctMe6lc7nqXYPzdAMYiLOgOS6XnbzI+uA292TBX7B3Rt0pCrKJF0gukNCIKR60v++kSPfPKCHQEHl1Dc+DbobvG5sgUupjHOhDxBqSvIg2EFa5H9iiHlKgDKni3MESQjpyW7luxPhoWbkY7Z5sVyj/fXTd3QqecFoNgQjfqgoCSHVu26pn770tpA91vQSwXpMMuj/D/OGzYUIrnLegaiXeV2YMOgycQq39BNfrbOa55DwHUxzCjIT6+fafohqf1qxskcdgGr/2sl59pXF8LMCex1IhXrbOcNhLvL+c4VRuqk6Lmfo9zSG0ecDNW4ZsjE2HTDvaGbIdPB0mMsVvnfJx4Cj76RCcEErDy04dDpMoynNwqz/m7WUJ/dwjhSJJzJjHg4aEs2OIkkSmw8v6Q79JoHkCSoIBAq+b32724exwkemUHnjFB9Lh4OQIFu2HvJQN4otLa1T0d104eBiv8m1I6NM9B/dCiDCrJ8GPKJBahq85V8fmWT1eYHXD6KmNhOsK/stJSFCk++fN6keNqaT3oqrWuWxHcTNUjkVTMPDhdNUxN3eG4axjM3CePSlGHFE+YFxsrtbuXW1wIv302n4xPpBogWevSxzsD/RQ2j9H+PoALctBASTL6LeUPsFXQvwfxVVh7/7Y23Yng6fh4EcyYa1ehsU7579Lgq57sD+MuthOdjR5L14+CZ7WrxeLtMEqcj187p6o665TS2awRFMtHNU2Al1PvWnX3IJmg6iNLgb0VYwn17Mxdu0TWsxZtiVO3OErsu55ZKUCG6Mn1xu2G0vyd4jDnNtqiu60ejH90T8pY+dCRxmkiNkn9BQRwpOQtG4qgrXbJeX2p8PDGWtTSXDdbIDqmJFeexFEmamDALdowb+Ujrx0Czv5Bo0uhpOoDiFhW4sLJTKX5+Ol/oh/B+k1b/EnEjxl52GQpkh1T7QYVPLq5W/lDNJSSxC6m6wMypXlC9V/nHzCbe1WZme+2uCx1dRE8dvyqpDKrolG/Qy+Gdi9c0Bm+/EkbN7s46KWM5+pMwiyE+ykmU7pb+DmgFUwmG+XGsbBF/Bb7ZQnhmYj4LMgiP29hh/D6+I6NR1xppGGOJ3ktlaJsCZVniShRyMTgdFphQn2s3VYWzy69gRC/cFHAC9LDqt+hL0tsvFkU6ffNMcxWaCQtrfqruuSXgFBFxk3lhP9XPHkP+S+ZLbVvqlpOrlbXF56CZQpfpMYxxRJXyIVFM9udj5lRoQ0FdLYnutcgrRECJHf18CqIRWHhzTwIMhJfWLE+OLMENURwm8tDNve38d7MAFiNb5PIVtlxqderAnaqzR+Yl3xzk6xfOwc+gQCb2xMLXU9Y0y8vrRMHDcwNndjOupGRQ33CmEsVuRe2MRs9wpOJiUwOVoioPTwlNsJJF3Pu5QwwEUvRWEu62rDfxr5koXSzHAdI0w/fp4dhi39mHybi4qr2mukwj5Nk8fHq08zElrxjwinNM1oNIoIiePrnS5/N0apSoqWrxhGUS1eVj1tImS+ykiD4Q4K/NLZXgBsiZRlghzF1KEiq59QMIbSWuUuWjkWaffiYyqyN2uc5gUnqi+AQvskPrfoMyabxpmcX6mKqFhvn+OVPZg8sIgVd7ffPXYAGxMtvoBaCyTzwXDAfQh+35XhosoQOy2vBVE2lVmjgqQj6Kzr9R3sapwRadfp+s2PuY6hZBS+ds/FTJQtKZajSG01thwWtMWcgKcQVWHj/+vzeFZ03kmovdb1i6P6h2N8jGWdPKoyaGS2DGyBaP71G3dbDFsixEoptdKbQ8grVUkuOh2QJzKS/ormJMDkpZZlbojs92gohF9W+wXieqdRZzxhveRoliC+elaQPLGRHLTnglLT8HUY49nXT2Y/+h1o+CPyhCLDqJJjFUitH5tC0QrykToBaDvkwkRhZt+h5YaopxKGqz6svhcZEdjzzHWZ1HdIIykZuwar38BCbvFAyH9T8F+3fODIkZiMW2HjKu0RCU5XqEZpilGDUN/NVElRAUb4cN5Oc0nVmJ4v4K1aUQzAneBTaiKSuDrArIuwEVtxd9BhUsOh1+8vW+8Qk8iXIi+ZaUfRNWsXTGyfVUOYI/ODNAE1Q9bQYK4Y+PNYWwUZvfDtaRfhKEgFq4FsqjA6BOGIu2Hlmghlh0VWaXI/ZgmrPwtaGyrhoBGZCOQHx8nHdkbFRKCgZpL6oKbmcDSO3KRxHXBThOdxLxjEUY8VtbdLhbBxBEJOdrJAp5wsHT7Ws7qpKIJ84lwCPstp/+1Jh+PmTJfkV0nqfW8gU121ULcf4dg9Eg+IbRJl2E0DoydYlFVAlkw8nwUtBze/oCvg2uSc4D8yzHC3uINt27KGCHl9X6UpNVV7nhJJy0XpR0j18pJbcwP5qp2Y0EeL0aaCBVZ/pMOwprdrcQCzabzHKTpHkOzfH3nmzT7T66CdMhhkR9EJZDQ7yxjMpkLEr38N9Q1wqpwA0sy8iWB2M4DCDJBlPBEPtV79ZbrO9o3fyH5UXoSaMNDRLEY3hI6y41iBj2PHriALQTNBKOrdfsWGMynQ3tbia+qFRngP1y2yHUbKkpeKMaeTfKaAv8qxKsD7LSVdX6YX1UMsP8+aYqmyaftGFAO7913FgpDSQakRfcYwxS3mytIIygVPGv9hCfeiAkNK0W5u1zIT9t338Hns+4lXKUe1bPR7+5S2A+Ag6AUavdOMMOc6h0lQPNoSOWCMoL8CgFTJ14JSRqVTbg5SHMF/f+gwVuZiI6aLDzWjEVzBt9oZhRVHSoSTWHpKSvnLOpYyEJoLjoa7M8yTQgZW3iXn9qrvvORvYhrZKYsNjd4syqWY7Rwbd0ETgpfgdKG4kF+ro9kqb7iRzAA32bcXwfyB5jre2fysvs2gHyEObuQvBtkRGqWItzRC0+URscjG7FGvTTfrZWUDNDY76JWd5AzorSRZi2ya+flta3dJVdVkdqD5qEr+W6awxYnAdsk8o/RQ7JgqnhB6+qGD8uB+BQE6n8wLIQ1rJXgEssOKN/EPzIzC394X/rYO0tqTZLpmBJmf1PwjL7ANPN8VPZpyoUzxiXYRQ9+3mFoSIKDNEcT1I8eSCkPyVxYBGhx+4llhaFvDAZTEucDbzsR+mGXqZKPzuuSONP4wPG9/ADbwi5ar33ZuNBvraE+O9iG9V0trL1v7lJDk/zxt+rf3j08I2EscuDI/IgjqKBB7R+bZ+GOflbzBlPBWXFld7JDkzDVxB17U7ME/LwKaTT56D2+GmHpePktGRLHwayskGpAChSogiYKH7Jlw9x3c9ckXpPlQdZgBEIM8kHCMUOB6k/NgXibTx57JsTkkGGx21N6Y0QvjjSlde4IS3V5hkuBGiRvTTjMrbXMdD/9u4h1UFRJIpt102ZIahrsWUnIOQ9gEhUJn9mnYFIkO/m/W4EOu6gBVYxNOJfQSGG1UE2JwAzOkIMKA/KFOmRX42PJ+0ruAviJs7XKbjudEkeQdLw4R+CgDj4pEBf81mOgXtbn1C0RjUfvYfeD/81otM9sLfFtYYfF/cu2GxSPUB4YY1qvb87HQbjBA9Ex4ufcf6kg36Lz9x5X8QkOUxPir6rUY/TGGnFLodN8vggT5ivf27MiZx/t8HwamjoRKI78ByU/YydR4oV+umBlz3qhNsAcKZMtCWSRuMEihhB5ijh9/uM1KtYfzgEtSxY9OPNDOtpXCr1BoO8ya1mYxyouR6o1EKNzVtpZH3sFBcn8aUS6JexJAOukWlZEOlSkEu9YbahOoDEy3uQdn7LUzf9SZxFVrNKuKf5C4TxKntw0tcfv3+ggOPDpfJA61yrLzsimV4mBNjNJDmo6lBiWbjE1qt/3kTugzrZBIGCdl6xGgd/vNgQPrgcsffwqyjDBQfyt6eqgXcmztyP/38YJw6uqOGx4BAEkJl08Kw3ARyR0OR86zLewXJsx8H1x/ssSXnk7ZgqnjfVGMfsESrpUYQCFr5eGcTDzJ3Llp2+Wu+/Vk6ZPsY5UbWZ8LiPTnj4hDOg15STR0Hson8P+77rqqKje+03mgFEqcZtlM8pJVx0uKCRpdBzt89tdqFdT1q18Jq5lOGCSelgggj9Iq3g9yO5StZlRt2Lt0/5PmVkbun4VFEd0anrNoAyEyGYBju0a/mhFAWIz53UIxmbFk2+sR+W/ZF8fIzVBmtuTmhDWb4Mi0PL1vk96KpH37wSYw+CWnhf6PwSTXnxWFBWv6JnjP1/i6SI7kJvRDf+Cs+86RzhLroGLkcQqUptEmcu35HjJLKHUsQamULjuPGC81i7g0gN4ZoLEoPAV5BbLQY7IfYL7CFNVG75/GpbMqlwCPrtzKIXxVVTVjKXr826z7olMx+Z1fQGROlkxepdyF7sZ9DMVYEHRET2hqshSeHIvbUW5QRg3RJpQBCZXbv3mimOOVVq2v+L5j2Bxa5wEoHKb0MVN6X6EL4rqzACZw2Mk1hjv9ejjVUzCyQaQpbWAzKsmmAVvKc4w74oqnbKnj54hzh8dr2guTbis0LEckxmlAkCypWWVwQpJLIVf6UiXYlypMXho/dV36NR8cQL0nWUhKL+8C7ASSX9WPYf/MHGBGHgVuENNlOmgsDcCQvwefsTcMeThAw5a38t7PYwxbZ0VO1qXM184d2+3QEoK3Imwb0V9UYIG1cT1a9NPVHJi4Y+WkZC69NKAr/Xc/BQlTT3Agm1jx9lHXmzY6dsnEiD5Fs2sOX38Fahb5Uwi9Nbm7nNo3nGmj3nsNUPLPuluNbSjWC7Z7L1OsZ2HgzRdQh0+TcIbu3EJX2ZBKdxBCT9WT41gL8FuBPv5PuhKlGeMFODwannZ2BtE/67VmmH5boYjE9Bfd1qC5w3wXhSYW+TJXfNXwWqfj+u6IXjKbJlsKOVap/xjPRxdWxYidlG6r0G7+Pxl+bQiofRvUqPwK4xIZczBQkSNPiCQTw3d0bky8aAJAkfXZXHIhg3uoiM+coQElqs+CUQ6UigtEHxyTEJ8cK2DXL0s3hK77/joZ6W8vSnPSKKTU0NFt84ocPCGHileDwLSRQ+PDDpQlfNpxa2u7cx3v09YeXrWhFB/WGaXJOcsnR67eMhRfHuJgdY//fTnv8FAktMWUh0/Dsov0i9gZxK4f2RAwiNj+Ut6yatGnyXZnXeY3y0qWg75qocQdUF/Iv3sBW08a+skk+EVRtQhpTXS9ALEnI0zoTND9/9CUysK31y2vBy37cDnIxvCRLCsSCZL84PFETOpkpHZ3Inp6NWElCqUElJhkUl8lJhAN7MdZh+WqnpwvmKzAjSx+GJ3k/4aPgCP5StVGeTSpN3dLOj3Dl+LFSMJbPpWAGln7A+fTWGbfN15N/nRarT05zHNn0Wn2RkwmL9F352wmhSR4L/WBUvvrVGMNaW8pysn8Qw/eVvEBj872JBJcy7LECcLhF5T9/NYZ0n54bITdBl6CgvscJdGOOtWRr3R/213R2Tqud+PSqLhRDyGjDmUZPC9RKeLfmCRqKlfsOSI2E7yezJKld3Bf7hRLRzlUKpnlBazhJpLc/Y9+NRNLKhyr+P3lQk8elvWqLXrumnIhGz97ime4T2sUCzOFR6j1adHxE1/hEguFG0TwHsPCwcREujw4vetWNyClMpQ2mlX6pkbQ9M0JX7TMlhi8rUtR3p//odvnh3mrFcwNcJeit+jbZLroSzsAjW7HjsEelkCIzHP7bRkFR52+yYgVlnQ5YrO+xrFStuSfsLwCL2h3IlhYdDrGweFh2Qf7BT1EVy8l7KWsOKNYnhoifzAe+Yk/kQGsE0H66/zC3wkIK2bG9qJpBk9G0AYRX2ORmfEw9d5t5hJm+LXneXnTuHkYaZL1qjk9NYZQs0s39iq7MTlO6JCXKK3gyBDz8SvG7F6rS/8eNi5SgHLozJ0EYg0vYU0ezRWyisyBRQEnyHcBkgm/4ds1L6NkdVCRhw7TjUcqceCSEhnpTrIfZ9E207AiZt3hgnBg/cKlYE4feuojNRLlQWX3O6RvLITlrgz+yXNI6B1ffXMpf0g4a1DzqAU4bgLNUUJ2lgQdnZTplziUDYQjvVvQs3sWPWtCnxj752aHSNecquUWqYBTlTz5BCVbjsaKCZYKrovbzlabXOxJexF0M+ziqFs1i5eG2nCtHYgArRVAfpGnMpnJOUfDU8ka5wWVtVitnYAR9zJOYumyzyJkvHANUADQ2/5dLH/cHkDDpPI7CL+GqAZrhGoJnQgdmpQ5+IFNKN+JrwA+fyGTQkc5OmaE57pMdUXScj2fo3RovTn+L7i5UyPZwvvxvW0QTaOa+bDBFcgPouQss9m5IgubNVlVqz7SB+KtJbwFkI0HiuPhs7wFaYOYb13fFDCJGyAnh2ri7bHUznK3MYYpZy8PGIPSvfrVlKoIOl5eRUChbyZjfD1I4MTdeEJsuxrO2CgymgAi1suo0BwLtun2IrG8428CNS9rm8/6wxDizh6aP17OZTV+KLEKdPtxd3xeTBbm1nrY5lVjCIqlTPNeR8VT/VfNekJ7P4KrbMKfxn98P15aFmhleimJeMPO227c3vs8gLlqrIhgF/fy1okmCquV2NgEw8dNa1qs61mhnAn9bloo43O+AfAeljZStRXVNRE2Lh9b8dkujaAchvn+62u2oXwy2QT4nUueOyUWF1IleOWomTvCSPzSJxSDY2enrzAsR3VrX0fSR2UO/lC6KNJT6Vk02W+jFo7wJ/RyqE2xiNMkGRW31IWYSFkuhTv1LmJCf3P34wSJ2OQ59Jg784Xn+0aTTmmt2mI6eAKS6QRVF9ccFMw8Znp3ajlCeafHtFeJS+1tEXH0HnRrovmJu3j0GxZ7RVdoPMp2LQ1udTYUyNDgMN861XKJb4dmV/mgZguzArOvx/EBh5qqz3Pc9FEtLeVW4WsUvYfW/s0fQh1uF8ZXoJHO4gY4xkrkN+tZnzR29VwmkfPQ9PmFjEWZc/UXBvoL50HLLHC0pe2yNxnu8bs9OPEsyy3IZuOmvq92Q9oeDtrsiv7lsQ1NCnnjveWYzx18yihwjfVgvMNZiU3sB/qYmW9SLYmoqWkTNqfdaVg6oH7BWAioRZtM8TvJvszlaGA5eh0I5hh9XEyToBctAv9k0shwh/TFHQfydC78viv1i36j7L0LN1+k8JiaZ84AnY5lVsMp99SJfiCQ1IHCzWt+w==",
'ctl00$CphCuerpoPagina$txtNombre':"",
'ctl00$CphCuerpoPagina$txtDireccion':"",
'ctl00$CphCuerpoPagina$txtCodigoPostal':"",
'ctl00$CphCuerpoPagina$ddlCCAA':"15",
'ctl00$CphCuerpoPagina$ddlProvincias':"31",
'ctl00$CphCuerpoPagina$ddlMunicipio':"0",
'ctl00$CphCuerpoPagina$txtActividad':"",
'ctl00$CphCuerpoPagina$ddlSubTipoOperador':"0",
'ctl00$CphCuerpoPagina$ddlActividadNivel1':"0",
'ctl00$CphCuerpoPagina$ddlCCAAUbicacion':"0",
'ctl00$CphCuerpoPagina$ddlProvinciasUbicacion':"0",
'ctl00$CphCuerpoPagina$btnBuscar':"Buscar"
}

In [63]:
resp = requests.post(base_url, params=params)

In [64]:
resp

<Response [200]>

In [65]:
html = resp.text

In [66]:
soup = BeautifulSoup(html, 'html.parser')

In [67]:
soup.find('div', attrs={'class':'sitio'})

<div class="sitio">
<ul class="bienvenida">
<li class="primera activa"><a href="https://servicio.mapama.gob.es/es" title="Castellano"><span lang="es" xml:lang="es">Bienvenidos</span></a></li>
<li class=""><a href="https://servicio.mapama.gob.es/ca" title="Català"><span lang="ca" xml:lang="ca">Benvinguts</span></a></li>
<li class=""><a href="https://servicio.mapama.gob.es/gl" title="Galego"><span lang="gl" xml:lang="gl">Benvidos</span></a></li>
<li class=""><a href="https://servicio.mapama.gob.es/eu" title="Euskara"><span lang="eu" xml:lang="eu">Ongi etorri</span></a></li>
<li class=""><a href="https://servicio.mapama.gob.es/va" title="Valencià"><span lang="ca" xml:lang="ca">Benvinguts</span></a></li>
<li class=""><a href="https://servicio.mapama.gob.es/en" title="English"><span lang="en" xml:lang="en">Welcome</span></a></li>
<li class="ultima"><a href="https://servicio.mapama.gob.es/fr" title="Français"><span lang="fr" xml:lang="fr">Bienvenues</span></a></li>
</ul>
<ul class="enlaces">

In [68]:
soup.find('table')

In [69]:
tabla = soup.find('table', attrs={'id':'CphCuerpoPagina_GVCertificados'})

In [70]:
tabla

In [71]:
html

'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head><title>\r\n\r\n</title>\r\n  \r\n    <link id="ctl00_idCss" rel="stylesheet" type="text/css" href="https://servicio.mapama.gob.es/es/_includes/css/style.css" media="screen" />    \r\n    <link type="text/css" href="../css/styleRegoe2.css" rel="stylesheet" /></head>\r\n<body class="home">\r\n\r\n    <div id="contenedor_pd" class="interior">\r\n        <div id="contenido_pd">\r\n            \r\n<div id="header">\r\n    <div class="nombre">\r\n        <a title="Ministerio de Agricultura y Pesca, Alimentación y Medio Ambiente " href="https://servicio.mapama.gob.es/es/"\r\n            accesskey="0">\r\n            <img src="https://servicio.mapama.gob.es/imgplantilla/bg_logo.png"\r\n                alt="Ministerio de Agricultura y Pesca, Alimentación y Medio Ambiente " />\r\n        </a>\r\n        <h1 class="n

In [7]:
from selenium import webdriver
import geckodriver_autoinstaller



geckodriver_autoinstaller.install()  # Check if the current version of geckodriver exists
                                     # and if it doesn't exist, download it automatically,
                                     # then add geckodriver to path

driver = webdriver.Firefox()

In [8]:
driver.get(base_url)

In [225]:
cp_input = driver.find_element_by_name("ctl00$CphCuerpoPagina$txtCodigoPostal")

In [226]:
cp_input.send_keys('02002')

In [227]:
driver.find_element_by_id("CphCuerpoPagina_btnBuscar").click()

In [235]:
tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")

In [10]:
import time

In [11]:
import tqdm

In [12]:
def get_rows(n):
    tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")
    rows = tabla[0].find_elements_by_xpath("//*[@class='GridViewRowEsilo' or @class='GridViewAlternatingRowEstilo']")
    df = pd.DataFrame([[td.text for td in row.find_elements_by_xpath(".//td")] for row in rows])
    try:
        links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")
        num = len(links)
        links[n].click()
    except:
        pass
        
    return df

In [13]:
total_pages = 0
total_records = 0
pages = []
for cod in tqdm.tqdm(cp['codigopostalid'].unique()):
    driver.get(base_url)
    cp_input = driver.find_element_by_name("ctl00$CphCuerpoPagina$txtCodigoPostal")
    cp_input.send_keys(cod)
    driver.find_element_by_id("CphCuerpoPagina_btnBuscar").click()
    # time.sleep(1)
    tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")
    if tabla:
        
        try:
            links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")
            num = len(links)
        except:
            num = 1
           
        for i in range(num):
            df = get_rows(i)
            pages.append(df)
            
        total_pages +=num
        print(f'cp:{cod}: {num} pages')

print(f'{total_pages} pages')



  0%|                                                      | 2/11039 [00:09<15:59:28,  5.22s/it]

cp:05480: 1 pages


  0%|                                                      | 15/11039 [00:30<5:40:59,  1.86s/it]

cp:01015: 1 pages


  0%|▏                                                     | 50/11039 [01:25<5:22:11,  1.76s/it]

...

cp:50840: 1 pages


100%|██████████████████████████████████████████████████▉| 11027/11039 [6:14:16<00:23,  1.93s/it]

cp:51001: 1 pages


100%|███████████████████████████████████████████████████| 11039/11039 [6:14:36<00:00,  2.04s/it]

4732 pages


In [275]:
df = pd.concat(pages)

In [282]:
df.drop([0,1], axis=1, inplace=True)

In [284]:
df.columns = ['Razón social','Dirección sede social','Provincia','Comunidad Autónoma','Actividad','Ubicación actividad','Grupo de productos','Entidad de control']

In [285]:
df.to_csv('REGOE_2021.csv')

In [76]:
sel = driver.find_element_by_name("ctl00$CphCuerpoPagina$ddlProvincias")

In [85]:
from selenium.webdriver.support.select import Select

In [104]:
selCCAA = Select(driver.find_element_by_xpath("//select[@name='ctl00$CphCuerpoPagina$ddlCCAA']"))


In [105]:
selCCAA.options

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="8fb4d78e-7876-423d-ae1f-f213e1cfb237")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="d19cdc5a-a46c-4263-a275-fe5a8d345b42")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="a1000c38-a019-4e84-aa52-2506dd9bd97c")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="cb0ca262-d969-4ad0-a62d-32f9b9f133f7")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="9468c4fc-797d-4e57-a366-9fc4c62a1a59")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="5497912a-cf02-44e8-b968-73c674f4e2c9")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement 

In [115]:
time.sleep(2)

NameError: name 'time' is not defined

In [110]:
selCCAA.select_by_visible_text("Andalucía")

In [111]:
selProv = Select(driver.find_element_by_xpath("//select[@name='ctl00$CphCuerpoPagina$ddlProvincias']"))


In [114]:
[opt.text for opt in selProv.options][1:]

['Almería',
 'Cádiz',
 'Córdoba',
 'Granada',
 'Huelva',
 'Jaén',
 'Málaga',
 'Sevilla']

In [116]:
selProv.select_by_visible_text("Granada")

In [117]:
driver.find_element_by_id("CphCuerpoPagina_btnBuscar").click()

In [188]:
tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")

In [189]:
tabla

[]

In [135]:
tabla[0].find_elements_by_xpath("//table/tbody/tr")

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="da617dd0-0a46-466d-830a-5c13324b776c")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="f22b4f88-65c9-4a62-9651-56a61c788525")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="26af10f7-063a-4cc7-b11a-e3635c1693f6")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="3b77a4bf-4119-4113-9dbb-e42ae0b4e2c8")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="3b26eb82-8b45-4960-b196-f8957d545812")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c35ee3e4-a7a0-4e75-9c32-f9033151aa67", element="4d4642af-b9d2-41ec-bc1c-394a572ecd1f")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement 

In [149]:
links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")

In [151]:
links[1].get_attribute('href')

"javascript:__doPostBack('ctl00$CphCuerpoPagina$GVCertificados','Page$1')"

In [153]:
links[1].click()

StaleElementReferenceException: Message: The element reference of [object String] "{\"element-6066-11e4-a52e-4f735466cecf\":\"b0ea812b-826a-41d2-8bb2-cfc2027cda7b\"}" is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


In [137]:
tabla[0].get_attribute('innerHTML')


'\n\t\t\t<caption>\n\t\t\t\tREGISTRO GENERAL DE OPERADORES ECOLOGICOS (REGOE)\n\t\t\t</caption><tbody><tr class="GridViewCabecera">\n\t\t\t\t<td class="GridViewColumnEstiloAncho" colspan="2">Visualizar pdf</td><td class="GridViewColumnEstiloAncho" colspan="5">Datos Generales del Operador</td><td class="GridViewColumnEstiloAncho" colspan="4">Datos de Actividad</td>\n\t\t\t</tr><tr class="GridViewCabecera">\n\t\t\t\t<th class="GridViewColumnEstilo" scope="col" abbr="CertificadoES">&nbsp;</th><th class="GridViewColumnEstilo" scope="col" abbr="CertificadoEN">&nbsp;</th><th class="GridViewColumnEstiloAncho" scope="col">Razón social</th><th class="GridViewColumnEstiloAncho" scope="col">Dirección sede social</th><th class="GridViewColumnEstilo" scope="col">Provincia</th><th class="GridViewColumnEstilo" scope="col">Comunidad Autónoma</th><th class="GridViewColumnEstiloAncho" scope="col">Actividad</th><th class="GridViewColumnEstilo" scope="col">Ubicación actividad</th><th class="GridViewColumn

In [ ]:
# identify dropdown with Select class

#select by select_by_visible_text() method
